<a href="https://colab.research.google.com/github/zakariazemmahi/waste-detection-yolov8/blob/main/Application_de_comptur_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **⚙️ Étape 1 – Installer les dépendances**

In [2]:
!pip install streamlit ultralytics
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

# **📝 Étape 2 – Créer ton fichier app.py Streamlit**

In [3]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image
import numpy as np
import requests
import streamlit.components.v1 as components

# ------------------------
# Configuration de la page
# ------------------------
st.set_page_config(
    page_title="SmartWasteDetection",
    page_icon="♻️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ------------------------
# Animation Lottie
# ------------------------
def load_lottie_url(url):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

lottie_url = "https://assets7.lottiefiles.com/packages/lf20_qp1q7mct.json"
lottie_json = load_lottie_url(lottie_url)

# ------------------------
# Barre latérale
# ------------------------
with st.sidebar:
    st.image("https://cdn-icons-png.flaticon.com/512/10067/10067108.png", width=80)
    st.title("♻️ Smart Waste Detection")
    st.markdown("Ce projet utilise **YOLOv8** pour détecter les objets, filtrer les déchets et classifier leur type.")
    st.markdown("---")
    st.markdown("👨‍💻 Réalisé par : **Zakariae Zemmahi Mohamed hajji Ess_aidi Youssef**")
    st.markdown("📍 Projet de fin de module")
    st.markdown("📅 Mai 2025")

# ------------------------
# Titre principal
# ------------------------
st.markdown("<h1 style='text-align: center; color: green;'>🌍 Smart Waste Detection</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center;'>Détection d'objet, filtrage de déchets, et classification du type (plastique, verre...)</p>", unsafe_allow_html=True)

if lottie_json:
    components.html(f"""
        <div style="display: flex; justify-content: center;">
            <lottie-player src="{lottie_url}" background="transparent" speed="1" style="width: 200px; height: 200px;" loop autoplay></lottie-player>
        </div>
        <script src="https://unpkg.com/@lottiefiles/lottie-player@latest/dist/lottie-player.js"></script>
    """, height=220)

# ------------------------
# Chargement des modèles
# ------------------------
model_detect = YOLO("/content/drive/MyDrive/yolov8_best_smartdetection.pt")
model_classify = YOLO("/content/drive/MyDrive/yolov8_best.pt")

# ------------------------
# Upload et traitement
# ------------------------
st.markdown("### 📤 Importer des images")
uploaded_files = st.file_uploader("Choisir une ou plusieurs images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        st.markdown("----")
        st.markdown(f"### 🖼️ Image : `{uploaded_file.name}`")
        image = Image.open(uploaded_file).convert("RGB")
        img_array = np.array(image)

        st.image(image, caption="Image originale", use_column_width=True)

        # Détection déchet / non-déchet
        result_detect = model_detect.predict(img_array, conf=0.25)[0]

        is_waste_detected = False
        boxes_waste = []
        for box in result_detect.boxes:
            cls_id = int(box.cls)
            name = result_detect.names[cls_id]
            if name == "dechet":
                is_waste_detected = True
                boxes_waste.append(box.xyxy.cpu().numpy().astype(int)[0])

        image_annotated = result_detect.plot()

        if is_waste_detected:
            st.success("♻️ Déchets détectés ! Classification en cours...")
            for bbox in boxes_waste:
                x1, y1, x2, y2 = bbox
                cropped = img_array[y1:y2, x1:x2]

                result_classify = model_classify.predict(cropped, conf=0.25)[0]
                for cbox in result_classify.boxes:
                    class_id = int(cbox.cls)
                    class_name = result_classify.names[class_id]
                    st.markdown(f"🔎 <span style='color:green'>Type de déchet détecté :</span> **{class_name}**", unsafe_allow_html=True)
        else:
            st.warning("🚫 Aucun déchet détecté. L’image ne sera pas analysée davantage.")

        st.image(image_annotated, caption="🧠 Résultat YOLOv8", use_column_width=True)

st.markdown("---")
st.markdown("<footer style='text-align: center; font-size: 12px;'>© 2025 - SmartWasteDetection - ENSAM Meknès</footer>", unsafe_allow_html=True)


Writing app.py


# **🌐 Étape 3 – Obtenir ton IP publique (optionnel)**

In [4]:
!wget -q -O - ipv4.icanhazip.com


34.16.147.252


# **🚀 Étape 4 – Lancer l'application et ouvrir un tunnel**

In [5]:
!streamlit run app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇

⠏your url is: https://loose-spoons-report.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.147.252:8501

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2025-06-05 18:17:01.182 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/d